<a href="https://colab.research.google.com/github/Ava-creates/decision_tree/blob/main/Decision_Tree_ID3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import csv
import math
import copy
import time
import numpy as np
from sklearn.model_selection import KFold
from google.colab import drive


In [ ]:
# reading from the file using numpy genfromtxt with delimiter ','



# normalize the entire dataset prior to learning using min-max normalization 
def normalize(data):
    print("normalizing the entire dataset")
    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    scaler.fit(data)
    X = scaler.transform(data)
    
    return X



In [ ]:
# Class node and explanation is self explanation
class Node(object):
    
    def __init__(self, level, leaf, label=None, attr=None, threshold=None):
        self.attr = attr # attribute to split on
        self.threshold = threshold # threshold of that attribute
        self.level = level # level of this node in the tree
        self.is_leaf = leaf
        self.label = label ##
        self.left_node = None
        self.right_node = None

    # method to identify if the node is leaf
    def is_leaf(self):    
        return self.is_leaf
        
    # method to return threshold value
    def ret_theta(self):
        return self.threshold
    
    # method return root value
    def ret_attribute(self):
        return self.attr
    
    # method return left tree
    def ret_lnode(self):
        return self.left_node

    # method return right tree
    def ret_rnode(self):
        return self.right_node

    def __repr__(self):
        if self.is_leaf:
            return "(Label: %r)" %(self.label)
        else:
            return "(Attribute: %r, Theta: %r, Left: %r, Right: %r)" % (self.attr, self.threshold, self.left_node, self.right_node)

In [ ]:
class DecisionTree(object):
    
    def __init__(self, X):
        self.root_node = None
        self.features = [x for x in range(X.shape[1])] # features
        print("New Decision Tree: Features--> " , self.features)
        self.used = [False for x in self.features] # whether the feature is used or not
        
        # stats of tree
        self.node_count = 0
        self.max_level = 0
        self.leaf_count = 0
       # self.verbose = verbose idk what it is
    
    def feature_depleted(self): 
        #check if all features have been used to build a tree
        for i in self.used:
          if(i==False):
            return False
        return True
        
    #fit function to start tree building process
    def fit(self, X, y, n_min):
        assert(X.shape[0] == y.shape[0])
        self.root_node = self.create_decision_tree(X, y, n_min)
        
        print('Decision Tree built.')
        print("Number of nodes:", self.node_count)
        print('Maximum depth:', self.max_level)
        print('Leaf nodes count:', self.leaf_count)
    
    # Iterative Dichotomiser 3 algorithm        
    def create_decision_tree(self, X, y, n_min, level = 0):
        # increase number of nodes created
        self.node_count += 1 

        #calculate maximum level of tree to display on debug screen
        self.max_level= math.floor(np.log2(self.node_count))

        #calculate major class label and frequency to store as a predicted label
        label, count = self.cal_major_class_values(y)

        
        
        if self.feature_depleted() or X.shape[0] <= n_min or count == len(y):
            # Stop splitting, when:
            # 1. no more features to split, or
            # 2. less samples in node than n_min, or
            # 3. the node is pure.
            
            #create lead node with specific level number and increase leaf count

            node = Node(level, True, label) #not sure what the label does in Node

            self.leaf_count += 1
            return node
        
        # find the best feature and threshold to split on
        feature, threshold = self.best_feature(X, y)
        
        #if no best feature or threshold found stop splitting the tree and create a leaf node
        if feature == -1 or threshold == -1:
            
            #create lead node with specific level number and increase leaf count
            node=Node(level, True, label, feature, threshold)
            
            return node
        
        # otherwise continue splitting
        
        #mark feature as used 
        self.used[feature] =True
        
        #create subtree root (subtree_root, with specified split feature and threshold)
        subtree_root = Node(level, False, label, feature, threshold)
        
        # split X, y for left and right subtree
        left_X= []
        right_X=[]
        left_y=[]
        right_y=[]
        for i in range(0, X.shape[0]):
          if(X[i, feature]<threshold):
            left_X.append(X[i, :])
            left_y.append(y[i])
          else:
            right_X.append(X[i, :])
            right_y.append(y[i])

        left_X=np.array(left_X)
        right_X=np.array(right_X)
        left_y=np.array(left_y)
        right_y=np.array(right_y)
     
        assert(left_X.shape[0]!=0 and right_X.shape[0]!=0)
        
        # build three recursively
        subtree_root.left_node = self.create_decision_tree(left_X, left_y, n_min, level + 1)
        subtree_root.right_node = self.create_decision_tree(right_X, right_y, n_min, level + 1)
        
        return subtree_root
        
    # find the feature and threshold that gives the most entropy gain
    def best_feature(self, X, y):
        best_gain = 10000000000000000000000000000000
        best_fea, threshold = -1, -1
        for i in range(0,len(self.features)):
            if(self.used[i]==False):
             thresholds = np.unique(X[:, i])
             for t in thresholds:
                gain = 0
               # parent_entropy =self.entropy(y)
                left_X= []
                right_X=[]
                left_y=[]
                right_y=[]
                for j in range(0, X.shape[0]):
                  if(X[j, i]<t):
                    left_X.append(X[j,:])
                    left_y.append(y[j])
                  else:
                    right_X.append(X[j, :])
                    right_y.append(y[j])

                left_X=np.array(left_X)
                right_X=np.array(right_X)
                left_y=np.array(left_y)
                right_y=np.array(right_y)

                n=len(y)
                ll=len(left_y)
                lr= len(right_y)
                enl= self.entropy(left_y)
                enr= self.entropy(right_y)
                children_entropy=(ll / n) * enl + (lr / n) * enr

                gain= children_entropy
                if(gain<best_gain):
                  best_gain=gain
                  best_fea=i
                  threshold=t


      
        return best_fea, threshold
                
    
    # find the dominant label in the node
    def cal_major_class_values(self, y):

        labels= np.unique(y)
        a=[]

        
        for i in labels:
          a.append(0)
        
        for i in range(0, y.shape[0]):
          for j in range(0 , labels.shape[0]):
            if(y[i]==labels[j]):
              a[j]+=1
        
        b=max(a)
        count=b
        dominant_label=-1
       
        for i in range(0, len(a)):
          if(a[i]==b):
            dominant_label= labels[i]
            break

        return dominant_label, count            
            
    # entropy calculation
    def entropy(self, y):
        y_ = y.astype(int)
        a = np.bincount(y_)
        b= a/len(y_)
        entropy= 0

        for i in b:
          if(i>0):
           entropy+= i*np.log2(i)


        return -1*entropy
    
    def predict(self, X):
        predicted = []
            #predict label for each example in input X starting from the root node
        for i in range(0,  X.shape[0]):
          node= self.root_node
          while (node.is_leaf==False):
            if(X[i, node.attr]<node.ret_theta()):
              node=node.ret_lnode()
            else:
              node= node.ret_rnode()

          predicted.append(node.label)
         
        return predicted

In [ ]:
#calculating the predicited accuracy
def accuracy(actual,predicted):
    right=0
    
    for i in range(0, len(predicted)):
      if(actual[i]==predicted[i]):
        right+=1

    accuracy= (right/len(predicted))



    # return the accuracy
    return accuracy

In [ ]:
def main(X, y, eta_min):
    #specify early termination criteria
    eta_min_val = round(eta_min*X.shape[0])
    
    #Normalize input data X
    X = normalize(X)

    total= []
    
    #generate 10 folds split on data
    kf = KFold(n_splits=10,shuffle=True, random_state=np.random.randint(10,21))
    
    for train_index, test_index in kf.split(X):

        X_train, X_test=X[train_index], X[test_index]
        y_train, y_test=y[train_index], y[test_index]
        tree= DecisionTree(X)
        tree.fit( X_train, y_train, eta_min_val)
        predicted=tree.predict(X_test)
        accu=accuracy(y_test, predicted)
        total.append(accu)
        
        #for each training and test fold build a decision tree and report accuracy

        print("Accuracy is ",accu)
    
    total=np.array(total)
    #print mean accuracy 
    mean =(np.sum(total)/10)

    sd= (np.std(total))
    #return mean accuracy of 10 folds and report standard deviation
    return  mean, sd

# Test on Iris Dataset

In [ ]:
import pandas as pd  # To read in the dataset we will use the Panda's library
df = pd.read_csv('./sample_data/Iris.csv', header=None, names =
                 ["sepal length[cm]","sepal width[cm]","petal length[cm]", "petal width", "label"])
df['label'] = df.label.map({'Iris-setosa': 0,
              'Iris-versicolor': 1,
              'Iris-virginica': 2})
X = df.values

y = X[1:,-1]
X = X[1:,:-1]
X=normalize(X)
print(X[0:10,:])



normalizing the entire dataset
[[0.22222222 0.625      0.06779661 0.04166667]
 [0.16666667 0.41666667 0.06779661 0.04166667]
 [0.11111111 0.5        0.05084746 0.04166667]
 [0.08333333 0.45833333 0.08474576 0.04166667]
 [0.19444444 0.66666667 0.06779661 0.04166667]
 [0.30555556 0.79166667 0.11864407 0.125     ]
 [0.08333333 0.58333333 0.06779661 0.08333333]
 [0.19444444 0.58333333 0.08474576 0.04166667]
 [0.02777778 0.375      0.06779661 0.04166667]
 [0.16666667 0.45833333 0.08474576 0.        ]]


In [ ]:
#testing with different early stopage criteria
eta_min_list = [.01, 0.05,0.10,0.15,0.20]

In [ ]:
df = pd.DataFrame(columns=['Eta min', 'Mean Accuracy', 'Sd'])
for i in eta_min_list:
    accu, std = main(X, y, i)
    print("----------------------------------------------------------------------------------")
    df = df.append({'Eta min': i, 'Mean Accuracy': accu, 'Sd': std}, ignore_index=True, sort=False)

normalizing the entire dataset
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of nodes: 9
Maximum depth: 3
Leaf nodes count: 5
Accuracy is  1.0
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of nodes: 9
Maximum depth: 3
Leaf nodes count: 5
Accuracy is  1.0
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of nodes: 9
Maximum depth: 3
Leaf nodes count: 5
Accuracy is  0.9333333333333333
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of nodes: 9
Maximum depth: 3
Leaf nodes count: 5
Accuracy is  0.9333333333333333
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of nodes: 9
Maximum depth: 3
Leaf nodes count: 5
Accuracy is  0.8666666666666667
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of nodes: 9
Maximum depth: 3
Leaf nodes count: 5
Accuracy is  0.9333333333333333
New Decision Tree: Features-->  [0, 1, 2, 3]
Decision Tree built.
Number of

### The result is summarized in the following table:

In [ ]:
df

,Eta min,Mean Accuracy,Sd
0,0.01,0.940000,0.046667
1,0.05,0.946667,0.040000
2,0.10,0.940000,0.055377
3,0.15,0.940000,0.046667
4,0.20,0.940000,0.055377


# Test on spam email dataset

In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
#eta_min_list = [0.75]

In [ ]:
df = pd.read_csv('/spambase.csv')
                 
X = df.values

y = X[:,-1]
X = X[:,:-1]
X=normalize(X)
print(X[0:10,:])

normalizing the entire dataset
[[4.62555066e-02 1.96078431e-02 9.80392157e-02 0.00000000e+00
  1.40000000e-02 4.76190476e-02 2.88858322e-02 6.30063006e-03
  0.00000000e+00 5.17051705e-02 8.04597701e-02 8.16959669e-02
  1.17117117e-01 2.10000000e-02 3.17460317e-02 7.00000000e-03
  9.80392157e-03 3.08030803e-02 1.85066667e-01 0.00000000e+00
  1.43114311e-01 0.00000000e+00 7.88990826e-02 3.44000000e-02
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.01596517e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.35356850e-02 0.00000000e+00 1.14539073e-02
  2.99850075e-02 2.42069696e-03 3.73490695e-03 1.00120144e-02
  6.48358586e-02]
 [1.32158590e-02 0.00000000e+00 1.39215686e-01 0.00000000e+00
  1.23000000e-01 3.23

In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)
X=X[0:200, :]
y=y[0:200]
df = pd.DataFrame(columns=['Eta min', 'Mean Accuracy', 'Sd'])
for i in eta_min_list:
    #running for 200 rows as it is too big
    accu, std = main(X, y, i)
    df = df.append({'Eta min': i, 'Mean Accuracy': accu, 'Sd': std}, ignore_index=True, sort=False)

normalizing the entire dataset
New Decision Tree: Features-->  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
Decision Tree built.
Number of nodes: 25
Maximum depth: 4
Leaf nodes count: 13
Accuracy is  0.8
New Decision Tree: Features-->  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
Decision Tree built.
Number of nodes: 29
Maximum depth: 4
Leaf nodes count: 15
Accuracy is  0.85
New Decision Tree: Features-->  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]
Decision Tree built.
Nu

In [ ]:
df

,Eta min,Mean Accuracy,Sd
0,0.05,0.835,0.070887
1,0.10,0.850,0.067082
2,0.15,0.845,0.082006
3,0.20,0.840,0.053852
4,0.25,0.850,0.074162
